What we need is a small parcel to work on, with a nice contourplot with critical points and minima found after initiating local method and then a 3d plot. 

In [1]:
using Pkg
Pkg.activate("../../.")
using Globtim
using DynamicPolynomials, DataFrames

  Activating project at `~/globtim`


In [2]:
# Constants and Parameters
const n, a, b = 2, 4, 12
const scale_factor = a / b   # Scaling factor appears in `main_computation`, maybe it should be a parameter.
const delta, alpha = 0.5, 1 / 10  # Sampling parameters
const tol_l2 = 3e-4            # Define the tolerance for the L2-norm
f = tref # Objective function

tref (generic function with 1 method)

One may assume that when we have access to exact evaluations, we would want to have a small $L^2$-norm tolerance `tol_l2 = 5e-4` and high probability of computing an accurate discrete $L^2$-norm `alpha= 1/10`.

We need to also return the number of samples used to generate the sample set. It is annoying that the error goes up while the degree has increased.

In [3]:
# rand_center = [2*rand()-1, 2*rand()-1]*.75; # Random center
# rand_center = [0.4694712315165298, -0.45637754560934185]
rand_center = [0.0, 0.0];

In [4]:
d = 30 # Initial Degree 
SMPL = 300 # Number of samples
TR = test_input(f, 
                dim = n,
                center=rand_center,
                GN=SMPL, 
                sample_range=scale_factor, 
                degree_max = d+4
                )
pol_cheb = Constructor(TR, d, basis=:chebyshev)
pol_lege = Constructor(TR, d, basis=:legendre);

@polyvar(x[1:n]); # Define polynomial ring 

current L2-norm: 0.00032540334035451827
current L2-norm: 0.21653386727580468


Solve the system of partial derivatives using `Msolve`. For degree `26` on tref centered at `rand_center = [0.4694712315165298, -0.45637754560934185]` and `a/b = 7//12`, `msolve runs single threaded in 5m 2.2 s. With 10 threads, it runs in 4m 48.2 s. 
The slow part of the process is the real root isolation it would seem. 

In [5]:
df_cheb = solve_and_parse(pol_cheb, x, f, TR)
sort!(df_cheb, :z, rev=true)


=== Starting MSolve Parser (dimension: 2) ===
Processed 269 points (0.002s)


Row,x1,x2,z
,Float64,Float64,Float64
1,0.0258173,-0.140992,6.28737
2,0.287448,0.170945,6.16155
3,0.286533,0.251541,6.09426
4,0.0274679,0.331984,5.83762
5,0.0274852,0.328735,5.83043
6,0.0274215,-0.288537,5.73714
7,0.286093,0.327728,5.43213
8,0.286086,0.332483,5.36644
9,-0.0774508,-0.139843,5.31879


Legendre polynomial: 8m49 s, `scale_factor = 1/3` and degree `30`, and 300 sample points.  

In [6]:
df_lege = solve_and_parse(pol_lege, x, f, TR, basis=:legendre)
sort!(df_lege, :z, rev=true)


=== Starting MSolve Parser (dimension: 2) ===
Processed 279 points (0.003s)


Row,x1,x2,z
,Float64,Float64,Float64
1,0.0259943,-0.140881,6.28934
2,0.287917,0.170829,6.16495
3,0.286793,0.251709,6.09483
4,0.0276141,0.331731,5.84256
5,0.0276342,0.327851,5.80976
6,0.027565,-0.288191,5.7418
7,0.286267,0.327073,5.42289
8,0.286258,0.33208,5.38275
9,-0.0777829,-0.139723,5.32538


In [20]:
using Optim
using LinearAlgebra
function analyze_critical_points(f, df_cheb, TR; tol_dist=0.025)
    # Initialize df_cheb columns
    df_cheb.y1 = zeros(nrow(df_cheb))
    df_cheb.y2 = zeros(nrow(df_cheb))
    df_cheb.close = falses(nrow(df_cheb))
    df_cheb.steps = zeros(nrow(df_cheb))
    df_cheb.converged = zeros(nrow(df_cheb))

    # Create df_min for collecting unique minimizers
    df_min = DataFrame(x1=Float64[], x2=Float64[], value=Float64[], captured=Bool[])

    for i in 1:nrow(df_cheb)
        try
            x0 = [df_cheb.x1[i], df_cheb.x2[i]]
            res = Optim.optimize(f, x0, LBFGS(), Optim.Options(show_trace=false))
            minimizer = Optim.minimizer(res)
            min_value = Optim.minimum(res)
            steps = res.iterations
            converged = Optim.converged(res)

            # Update df_cheb
            df_cheb.y1[i] = minimizer[1]
            df_cheb.y2[i] = minimizer[2]
            df_cheb.steps[i] = steps
            df_cheb.converged[i] = min_value

            # Check if minimizer is close to its starting point
            distance = norm([df_cheb.x1[i] - minimizer[1], df_cheb.x2[i] - minimizer[2]])
            df_cheb.close[i] = distance < tol_dist

            # Check if minimizer is within the square
            within_square = abs(minimizer[1] - TR.center[1]) <= TR.sample_range &&
                            abs(minimizer[2] - TR.center[2]) <= TR.sample_range

            if !within_square
                continue  # Skip this minimizer if it's outside the square
            end

            # Check if this minimizer is already in df_min
            is_new = true
            for j in 1:nrow(df_min)
                if norm([df_min.x1[j] - minimizer[1], df_min.x2[j] - minimizer[2]]) < tol_dist
                    is_new = false
                    break
                end
            end

            # If it's a new minimizer within range, add it to df_min
            if is_new
                # Check if it's close to any original point
                is_captured = any(norm([df_cheb.x1[k] - minimizer[1], df_cheb.x2[k] - minimizer[2]]) < tol_dist
                                  for k in 1:nrow(df_cheb))

                push!(df_min, (x1=minimizer[1], x2=minimizer[2],
                    value=min_value, captured=is_captured))
            end

        catch e
            # Set default values for error cases in df_cheb
            df_cheb.y1[i] = NaN
            df_cheb.y2[i] = NaN
            df_cheb.close[i] = false
            df_cheb.steps[i] = -1
            df_cheb.converged[i] = Inf
            continue
        end
    end

    return df_cheb, df_min
end


analyze_critical_points (generic function with 1 method)

In [21]:
df_cheb, df_min_cheb = analyze_critical_points(f, df_cheb, TR)
df_lege, df_min_lege = analyze_critical_points(f, df_lege, TR)

(278×8 DataFrame
 Row │ x1          x2          z         y1          y2          close  steps  ⋯
     │ Float64     Float64     Float64   Float64     Float64     Bool   Float6 ⋯
─────┼──────────────────────────────────────────────────────────────────────────
   1 │  0.0259943  -0.140881    6.28934  -3.79644     1.56576    false      7. ⋯
   2 │  0.287917    0.170829    6.16495   0.244239    0.368302   false      9.
   3 │  0.286793    0.251709    6.09483  -0.189043    0.913256   false     10.
   4 │  0.0276141   0.331731    5.84256  -0.293381    0.774042   false     11.
   5 │  0.0276342   0.327851    5.80976  -1.93051    -2.37499    false      9. ⋯
   6 │  0.027565   -0.288191    5.7418   -3.57164    -3.31482    false      6.
   7 │  0.286267    0.327073    5.42289  -1.16829    -3.63373    false      7.
   8 │  0.286258    0.33208     5.38275   0.341808   -0.008879   false      6.
  ⋮  │     ⋮           ⋮          ⋮          ⋮           ⋮         ⋮       ⋮   ⋱
 272 │  0.0747926   0.1

In [16]:
using GLMakie

function plot_polyapprox_levelset(pol::ApproxPoly, TR::test_input, df::DataFrame, df_min::DataFrame)
    coords = pol.scale_factor * pol.grid .+ TR.center'
    z_coords = pol.z

    if size(coords)[2] == 2
        fig = Figure(size=(1000, 600))
        ax = Axis(fig[1, 1], title="Trefethen Function Level Sets",
            xlabel="X-axis", ylabel="Y-axis")

        # Create a regular grid for contour plotting
        x_unique = sort(unique(coords[:, 1]))
        y_unique = sort(unique(coords[:, 2]))

        Z = fill(NaN, (length(y_unique), length(x_unique)))

        for (idx, (x, y, z)) in enumerate(zip(coords[:, 1], coords[:, 2], z_coords))
            i = findlast(≈(y), y_unique)
            j = findlast(≈(x), x_unique)
            if !isnothing(i) && !isnothing(j)
                Z[j, i] = z
            end
        end

        # Create contour plot
        contourf!(ax, x_unique, y_unique, Z,
            colormap=:viridis,
            levels=30)

        if :close in propertynames(df)
            not_close_idx = .!df.close
            if any(not_close_idx)
                scatter!(ax, df.x1[not_close_idx], df.x2[not_close_idx],
                    markersize=5,
                    color=:orange,
                    label="Far")
            end

            close_idx = df.close
            if any(close_idx)
                scatter!(ax, df.x1[close_idx], df.x2[close_idx],
                    markersize=10,
                    color=:green,
                    label="Near")
            end
        else
            scatter!(ax, df.x1, df.x2,
                markersize=2,
                color=:orange,
                label="All points")
        end

        # Plot uncaptured minimizers from df_min in red
        uncaptured_idx = .!df_min.captured
        if any(uncaptured_idx)
            scatter!(ax, df_min.x1[uncaptured_idx], df_min.x2[uncaptured_idx],
                markersize=20,
                marker=:diamond,
                color=:red,
                label="Uncaptured minima")
        end

        # Add legend to the right of the plot
        Legend(fig[1, 2], ax, "Critical Points",
            tellwidth=true)

        # Add colorbar
        Colorbar(fig[1, 3], limits=(minimum(z_coords), maximum(z_coords)),
            colormap=:viridis,
            label="Function value")

        display(fig)
        return fig
    end
end

function plot_polyapprox_rotate(pol::ApproxPoly, TR::test_input, df::DataFrame, df_min::DataFrame)
    coords = pol.scale_factor * pol.grid .+ TR.center'
    z_coords = pol.z

    if size(coords)[2] == 2
        fig = Figure(size=(1000, 600))
        ax = Axis3(fig[1, 1], title="Trefethen Function",
            xlabel="X-axis", ylabel="Y-axis", zlabel="Z-axis")

        # Create a regular grid for surface plotting
        x_unique = sort(unique(coords[:, 1]))
        y_unique = sort(unique(coords[:, 2]))

        Z = fill(NaN, (length(y_unique), length(x_unique)))

        for (idx, (x, y, z)) in enumerate(zip(coords[:, 1], coords[:, 2], z_coords))
            i = findlast(≈(y), y_unique)
            j = findlast(≈(x), x_unique)
            if !isnothing(i) && !isnothing(j)
                Z[j, i] = z
            end
        end

        surface!(ax, x_unique, y_unique, Z,
            colormap=:viridis,
            transparency=true,
            alpha=0.8)

        if :close in propertynames(df)
            not_close_idx = .!df.close
            if any(not_close_idx)
                scatter!(ax, df.x1[not_close_idx], df.x2[not_close_idx],
                    df.z[not_close_idx],
                    markersize=5,
                    color=:orange,
                    label="Far")
            end

            close_idx = df.close
            if any(close_idx)
                scatter!(ax, df.x1[close_idx], df.x2[close_idx],
                    df.z[close_idx],
                    markersize=10,
                    color=:green,
                    label="Near")
            end
        else
            scatter!(ax, df.x1, df.x2,
                df.z,
                markersize=2,
                color=:orange,
                label="All points")
        end

        # Plot uncaptured minimizers from df_min in red
        uncaptured_idx = .!df_min.captured
        if any(uncaptured_idx)
            scatter!(ax, df_min.x1[uncaptured_idx], df_min.x2[uncaptured_idx],
                df_min.value[uncaptured_idx],
                markersize=20,
                marker=:diamond,
                color=:red,
                label="Uncaptured minima")
        end

        # Add legend to the right of the plot
        Legend(fig[1, 2], ax, "Critical Points",
            tellwidth=true)

        record(fig, "trefethern_rotation_d30.mp4", 1:240; framerate=30) do frame
            ax.azimuth[] = 1.7pi + 0.4 * sin(2pi * frame / 240)
            ax.elevation[] = pi / 4 + 0.3 * cos(2pi * frame / 240)
        end

        display(fig)
        return fig
    end
end

plot_polyapprox_rotate (generic function with 1 method)

In [17]:
plot_polyapprox_rotate(pol_cheb, TR, df_cheb, df_min_cheb)

In [24]:
plot_polyapprox_levelset(pol_cheb, TR, df_cheb, df_min_cheb)

In [23]:
plot_polyapprox_levelset(pol_lege, TR, df_lege, df_min_lege)

In [25]:
# GLMakie.closeall()